<a href="https://colab.research.google.com/github/ksong1009/capstone/blob/main/Plotly_dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import required libraries
!pip install dash
import pandas as pd
import plotly.graph_objects as go
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px
import os

# Read the spacex_launch data into a pandas dataframe
spacex_df = pd.read_csv(
    'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv',
    encoding="ISO-8859-1"
)
launch_sites_data = spacex_df.copy()

# Extract unique launch sites
launch_sites = launch_sites_data['Launch Site'].unique()
dropdown_options = [{'label': 'All Sites', 'value': 'ALL'}] + \
                   [{'label': site, 'value': site} for site in launch_sites]

# Create a dash application
app = dash.Dash(__name__)

app.layout = html.Div(children=[
    html.H1('Spacex Launch Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),

    html.Div([
        "Select a Launch Site here: ",
        dcc.Dropdown(
            id='launch-site-dropdown',
            options=dropdown_options,
            value='ALL',
            placeholder="Select a Launch Site",
            style={'height': '50px', 'font-size': 20}
        ),
    ], style={'font-size': 20}),

    html.Br(),
    html.Div(dcc.Graph(id='piechart')),
    html.Div([
        html.H2("Payload Range (kg):"),  # Title for the slider
        dcc.RangeSlider(
            id='payload-slider',
            min=0,
            max=10000,
            step=1000,
            marks={i: str(i) for i in range(0, 10001, 1000)}, # More detailed marks
            value=[0, 10000]  # Initial range
        ),
        html.Div(id='payload-range-display')  # Output for displaying the range
    ]),
      html.Div(dcc.Graph(id='scatter-chart'))  # Add a scatter-chart here
])

# Callback for updating pie chart
@app.callback(Output(component_id='piechart', component_property='figure'),
              Input(component_id='launch-site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        # Show distribution of successful/failed launches across all sites
        fig = go.Figure(data=[go.Pie(labels=launch_sites_data['Launch Site'], values=launch_sites_data['class'])])
    else:
        # Filter data for selected site
        filtered_df = launch_sites_data[launch_sites_data['Launch Site'] == entered_site]

        # Count occurrences of each class (Success/Failure)
        class_counts = filtered_df['class'].value_counts().reset_index()
        class_counts.columns = ['index', 'class']  # Renaming for clarity

        fig = go.Figure(data=[go.Pie(labels=class_counts['index'], values=class_counts['class'])])

    fig.update_layout(title=f"Launch Outcomes for {entered_site}")
    return fig

# Callback to update the payload range display
@app.callback(
    Output(component_id='payload-range-display', component_property='children'),
    Input(component_id='payload-slider', component_property='value')
)
def update_payload_range(selected_range):
    return f"Selected Payload Range: {selected_range[0]} kg - {selected_range[1]} kg"

# Callback function for Scatter Plot
@app.callback(
    Output(component_id='scatter-chart', component_property='figure'),
    [
        Input(component_id='launch-site-dropdown', component_property='value'),
        Input(component_id='payload-slider', component_property='value')
    ]
)
def update_scatter_chart(selected_site, payload_range):
    min_payload, max_payload = payload_range

    # Filter dataset based on payload mass range
    filtered_df = spacex_df[
        (spacex_df['Payload Mass (kg)'] >= min_payload) &
        (spacex_df['Payload Mass (kg)'] <= max_payload)
    ]

    if selected_site == 'ALL':
        # Scatter plot for all sites
        fig = px.scatter(
            filtered_df,
            x='Payload Mass (kg)',
            y='class',
            color='Booster Version Category',
            title="Payload Mass vs. Launch Outcome (All Sites)"
        )
    else:
        # Filter for selected launch site
        site_df = filtered_df[filtered_df['Launch Site'] == selected_site]

        # Scatter plot for a specific site
        fig = px.scatter(
            site_df,
            x='Payload Mass (kg)',
            y='class',
            color='Booster Version Category',
            title=f"Payload Mass vs. Launch Outcome ({selected_site})"
        )

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(port=8051) # Changed port from default 8050 to 8051

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


<IPython.core.display.Javascript object>